In [22]:
from bs4 import BeautifulSoup
import requests as rq

In [5]:
get = rq.get('https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p=2').content

In [14]:
bs = BeautifulSoup(get)

In [16]:
bs

<!DOCTYPE html>
<html lang="ru"><head><link href="https://www.google-analytics.com" rel="preconnect"/><link href="https://yastatic.net" rel="preconnect"/><title data-react-helmet="true">Купить квартиру 🏢 в Москве: вторичка и новостройки | Цены на квартиры в Москве | Авито</title><meta charset="utf-8"/><meta content="telephone=no" name="format-detection"/><meta content="on" http-equiv="cleartype"/><meta content="499bdc75d3636c55" name="yandex-verification"/><meta content="7iEzRRMJ2_0p66pVS7wTYYvhZZSFBdzL5FVml4IKUS0" name="google-site-verification"/><meta content="https://m.avito.ru/icons/open-graph-default.svg" data-react-helmet="true" property="og:image"/><meta content="322" data-react-helmet="true" property="og:image:width"/><meta content="112" data-react-helmet="true" property="og:image:height"/><meta content="website" data-react-helmet="true" property="og:type"/><meta content="Авито" data-react-helmet="true" property="og:site_name"/><meta content="ru_RU" data-react-helmet="true" pro

### Функция для взятия ссылок на квартиры
На вход - страница поиска с квартирами в формате BeautifulSoup

In [249]:
def get_hrefs(page_bs):
    parts = page_bs.findAll('a', {'class': 
    "link-link-MbQDP link-design-default-_nSbv title-root-j7cja iva-item-title-_qCwt title-listRedesign-XHq38 title-root_maxHeight-SXHes"})
    hrefs = list(map(lambda x:'https://www.avito.ru' +  x.attrs['href'], parts))
    return(hrefs)
hrefs = get_hrefs(bs)
hrefs

['https://www.avito.ru/moskva/kvartiry/1-k._kvartira_459_m_825_et._2125363069',
 'https://www.avito.ru/moskva/kvartiry/3-k._kvartira_1015_m_1924_et._2210453521',
 'https://www.avito.ru/moskva/kvartiry/1-k._kvartira_579_m_1418_et._2267419075',
 'https://www.avito.ru/moskva/kvartiry/1-k._kvartira_345_m_2029_et._2258379115',
 'https://www.avito.ru/moskva/kvartiry/1-k._apartamenty_57_m_210_et._2264599000',
 'https://www.avito.ru/moskva/kvartiry/3-k._kvartira_871_m_313_et._2189600898',
 'https://www.avito.ru/moskva/kvartiry/4-k._kvartira_984_m_2026_et._2300570800',
 'https://www.avito.ru/moskva/kvartiry/kvartira-studiya_335_m_2032_et._2177892692',
 'https://www.avito.ru/moskva/kvartiry/1-k._kvartira_375_m_1728_et._2152975746',
 'https://www.avito.ru/moskva/kvartiry/1-k._kvartira_355_m_1021_et._2178410512',
 'https://www.avito.ru/moskva/kvartiry/2-k._kvartira_688_m_2339_et._2267014048',
 'https://www.avito.ru/moskva/kvartiry/3-k._apartamenty_841_m_1824_et._2267683734',
 'https://www.avito.ru

In [201]:
flat = rq.get(hrefs[0])

In [202]:
flat = BeautifulSoup(flat.content)

## Функция парсинга информации из веб страницы с квартирой
На вход - веб страница в формате BeautifulSoup. На выход - словарик с данными

In [203]:
def get_about_flat(flat_bs):
    params_list = flat_bs.findAll('li', {'class': 'item-params-list-item'})
    params = {p: None for p in ['Этаж', 'Количество комнат', 'Тип комнат', 'Общая площадь', 'Площадь кухни', 
                                'Жилая площадь', 'Отделка', 'Балкон или лоджия', 'Высота потолков', 
                                'Санузел', 'Вид из окон', 'Тип дома', 'Год постройки']}
    for param in map(lambda x: x.text.strip(), params_list):
        param = param.split(':')
        if param[0] in params:
            params[param[0]] = param[1].strip()
    params['Адрес'] = flat_bs.find('span', {'class', "item-address__string"}).text.strip()
    n_metro = len(flat_bs.findAll('span', {'class': 'item-address-georeferences-item__content'}))
    if n_metro > 0: 
        params['Ближайшее Метро'] = flat_bs.find('span', {'class': 'item-address-georeferences-item__content'}).text.strip()
        params['Расстояние до ближайшего метро'] = flat_bs.find('span', 
                                                        {'class': "item-address-georeferences-item__after"}).text.strip()
        params['Цвет ветки метро'] = flat_bs.find('i', {'class': 'item-address-georeferences-item-icons__icon'}).attrs['style'].split(' ')[1]
    params['Сколько станций рядом'] = n_metro
    return(params)

get_about_flat(flat)

{'Этаж': '8 из 25',
 'Количество комнат': '1',
 'Тип комнат': 'изолированные',
 'Общая площадь': '45.9\xa0м²',
 'Площадь кухни': '12.2\xa0м²',
 'Жилая площадь': '17.7\xa0м²',
 'Отделка': 'чистовая',
 'Балкон или лоджия': 'лоджия',
 'Высота потолков': None,
 'Санузел': None,
 'Вид из окон': None,
 'Тип дома': 'монолитный',
 'Год постройки': None,
 'Адрес': 'ул. Летная, вл. 95Б',
 'Ближайшее Метро': 'Спартак',
 'Расстояние до ближайшего метро': '700 м',
 'Цвет ветки метро': '#943E90',
 'Сколько станций рядом': 3}

### Будем собирать данные по рынку вторичного жилья

In [23]:
import pickle
import collections
import os
from tqdm import tqdm
import re
import pickle
import time
from selenium import webdriver
from fake_useragent import UserAgent

#### Сделаем прокси-лист для парсинга

In [213]:
proxy_url = "https://www.my-proxy.com/free-proxy-list.html"

driver = webdriver.Chrome(executable_path='C:/Users/DNS/Desktop/NIR/chromedriver')
driver.get(proxy_url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

proxtotal = list(soup.find('div', {'class': 'list'}))[::2][:-1]

driver.close()

with open('proxy.txt', 'a') as f:
    f.write('\n'.join(proxtotal))

In [214]:
with open('proxy.txt', 'r') as f:
    proxies = f.read()
proxies = [{'http': p[:-3]} for p in proxies.split()]
proxies[:5]

[{'http': '45.132.185.125:8085'},
 {'http': '83.171.254.207:8085'},
 {'http': '85.209.151.182:8085'},
 {'http': '154.85.100.95:5136'},
 {'http': '45.8.134.200:7216'}]

In [296]:
def get_bs(url, proxy, agent=UserAgent().chrome):
    resp = rq.get(url, proxies = proxy, headers={'User-Agent': agent})
    print(resp.status_code)
    if resp.status_code == 429:
        time.sleep(10)
        driver = webdriver.Chrome(executable_path='C:/Users/DNS/Desktop/NIR/chromedriver')
        driver.get(proxy_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        driver.close()
        return(soup)
    elif resp.status_code == 200:
        return(BeautifulSoup(resp.content))
    return(False)

#### Функция для переформатирования страницы в формат BeautifulSoup 

In [303]:
def get_bs(url, proxy, agent=UserAgent().chrome):
    resp = rq.get(url, proxies = proxy, headers={'User-Agent': agent})
    print(resp.status_code)
    if resp.status_code == 200:
        return(BeautifulSoup(resp.content))
    return(False)

#### Функция для извлечения ссылок на n_room-комнатные квартиры из n_pages страниц поиска

In [306]:
def get_All_hrefs(n_room, n_pages, proxies):
    url = 'https://www.avito.ru/moskva/kvartiry/prodam/' + str(n_room)
    n = 0
    All_hrefs = []
    for p in range(1, n_pages+1):
        if n_room == 'studii':
            page = url + '/vtorichka?p=' + str(p)
        else:
            page = url + '-komnatnye/vtorichka?p=' + str(p)
        page_bs = get_bs(page, proxies[n])
        while not page_bs:
            time.sleep(10)
            n += 1
            page_bs = get_bs(page, proxies[n])
            if n == len(proxies)-1:
                n = 0
        hrefs = get_hrefs(page_bs)
        All_hrefs.extend(hrefs)
        time.sleep(10)
    return(All_hrefs)

## Парсинг части квартир каждого "типа"
Всего объявлений для каждого типа квартиры указано в списке for_parse. Было взято 1/10 часть квартир каждого типа

### Сначала соберем все ссылки на квартиры

In [307]:
for_parse = [('studii', 10000), (1, 6000), (2, 9000), (3, 8000), (4, 3000), (5, 2000)]
part = 10
for_parse = list(map(lambda x: (x[0], x[1]//part//50 + 1), for_parse))
for_parse
for n_rooms, n_pages in for_parse:
    cur_hrefs = get_All_hrefs(n_rooms, n_pages, proxies)
    with open('flats_from_avito.txt', 'a') as f:
        f.write('\n'.join(cur_hrefs))
        f.write('\n')

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


### Теперь получим информацию по каждой квартире

In [162]:
def get_about_flat(flat_bs):
    params_list = flat_bs.findAll('li', {'class': 'item-params-list-item'})
    params = {p: None for p in ['Этаж', 'Количество комнат', 'Тип комнат', 'Общая площадь', 'Площадь кухни', 
                                'Жилая площадь', 'Ремонт', 'Отделка', 'Балкон или лоджия', 'Высота потолков', 
                                'Санузел', 'Вид из окон', 'Тип дома', 'Год постройки']}
    for param in map(lambda x: x.text.strip(), params_list):
        param = param.split(':')
        if param[0] in params:
            params[param[0]] = param[1].strip()
    try:
        params['Адрес'] = flat_bs.find('span', {'class', "item-address__string"}).text.strip()
    except:
        print('Не получен адрес')
    cond = flat_bs.find('span', {'class': "item-address-georeferences-item__after"})
    if not cond is None: 
        metro = flat_bs.findAll('span', {'class': 'item-address-georeferences-item__content'})
        params['Ближайшее Метро'] = metro[0].text.strip()
        params['Расстояние до ближайшего метро'] = cond.text.strip()
        params['Цвет ветки метро'] = flat_bs.find('i', {'class': 'item-address-georeferences-item-icons__icon'}).attrs['style'].split(' ')[1]
        params['Сколько станций рядом'] = len(metro)
    return(params)

def get_bs(url, proxy, agent=UserAgent().chrome):
    session = rq.Session()
    resp = session.get(url, headers={'User-Agent': agent}, timeout=2)
    print(resp.status_code, end = ' ')
    if resp.status_code == 200:
        return(BeautifulSoup(resp.content))
    return(False)

In [18]:
with open('flats_from_avito.txt', 'r') as f:
    all_flats = f.read()
all_flats = all_flats.split('\n')
print("Всего ссылок на квартиры - {:.0f}".format(len(all_flats)))

Всего ссылок на квартиры - 4101


### Для начала попробуем достать информацию с первой тысячи квартир

In [215]:
first_1000 = all_flats[:1000]

In [216]:
all_info = []
n = 0
count = 0
for f in first_1000:
    flat_bs = None
    while flat_bs is None:
        try:
            flat_bs = get_bs(f, proxies[n])
            count += 1
        except:        
            print('Прокси неудачный')
            proxies.pop(n)
            time.sleep(2)
            n += 1
            if n+1 >= len(proxies):
                n = 0
    if flat_bs != False:
        all_info.append(get_about_flat(flat_bs))
    if count >= 10:
        count = 0
        if n + 1 >= len(proxies):
            n = 0
        else:
            n += 1
    time.sleep(2)

404 200 Не получен адрес
200 200 200 200 200 200 200 200 200 200 200 Прокси неудачный
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Прокси неудачный
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 Прокси неудачный
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Прокси неудачный
200 200 200 200 200 200 200 200 200 200 

In [217]:
import pandas as pd
pd.DataFrame(all_info).to_excel(r'C:\Users\DNS\Desktop\flats_avito.xlsx', index = False, header=True)

### Теперь все остальное:)

In [218]:
rem_flats = all_flats[2583:]
all_info = []
n = 0
count = 0
for f in rem_flats:
    flat_bs = None
    while flat_bs is None:
        try:
            flat_bs = get_bs(f, proxies[n])
            count += 1
        except:        
            print('Прокси неудачный')
            proxies.pop(n)
            time.sleep(2)
            n += 1
            if n+1 >= len(proxies):
                n = 0
    if flat_bs != False:
        all_info.append(get_about_flat(flat_bs))
    if count >= 10:
        count = 0
        if n + 1 >= len(proxies):
            n = 0
        else:
            n += 1
    time.sleep(2)

200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 200 Не получен адрес
200 200 200 200 200 Не получен адрес
200 200 200 200 200 200 200 200 Не получен адрес
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 Не получен адрес
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 Не получен адрес
200 200 200 200 200 200 200 404 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200

Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачный
Прокси неудачн

IndexError: pop from empty list

### Финал - собираем общий датасет

In [228]:
df = pd.read_excel('C:/Users/DNS/Desktop/flats_avito.xlsx')
df2 = pd.read_excel('C:/Users/DNS/Desktop/flats_avito_2.xlsx')
df3 = pd.DataFrame(all_info)
final_df = pd.concat([df, df2, df3], axis = 0)
final_df.to_excel(r'C:\Users\DNS\Desktop\final_avito.xlsx', index = False, header=True)

#УРАА НАКОНЕЦ-ТООООО!!!!!